In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload
from tqdm import tqdm

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)


model = HookedTransformer.from_pretrained(
    "gpt2-small",

    fold_ln=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)


In [ ]:


# path = "../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/1100001280_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152.pt"
path = "../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152:v9/final_sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")



In [ ]:

log_feature_sparsity = torch.load("../week_8_jan/artifacts/sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152:v9/log_feature_sparsity_5000_4.pt")
# px.histogram(log_feature_sparsity, nbins=100, title="log_fe|ature_sparsity")
px.histogram(log_feature_sparsity[log_feature_sparsity>-9], title="log_feature_sparsity (excluding dead neurons)")

In [ ]:
from sae_training.utils import LMSparseAutoencoderSessionloader
model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)

In [ ]:
import webbrowser
from IPython.core.display import display, HTML

path_to_html = "../week_8_jan/gpt2_small_features_layer_5"
def render_feature_dashboard(feature_id):
    
    path = f"{path_to_html}/data_{feature_id:04}.html"
    
    print(f"Feature {feature_id}")
    if os.path.exists(path):
        # with open(path, "r") as f:
        #     html = f.read()
        #     display(HTML(html))
        webbrowser.open_new_tab("file://" + os.path.abspath(path))
    else:
        print("No HTML file found")
    
    return

# for feature in [100,300,400]:
#     render_feature_dashboard(feature)

## Does GPT2 do induction well on HP paragraph one?

In [ ]:
random_tokens, random_token_groups = generate_random_token_prompt(model, n_random_tokens=5, n_repeat_tokens=2, token_of_interest=" Mary")
prompt = model.to_string(random_tokens)
token_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([prompt], model, sparse_autoencoder, head_idx_override=5)
print(token_df.columns)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
token_df[filter_cols].tail(10).style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm",
)

In [ ]:
def plot_attn(patterns, token_df, title="", facet_col_labels = ["Original", "Reconstructed"]):
    '''
    # patterns_original = cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    # patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    patterns_original = cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
    both_patterns = torch.stack([patterns_original, patterns_reconstructed])
    plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")
    
    '''
    fig = px.imshow(patterns, text_auto=".2f", title=title,
                    facet_col=0,
                    color_continuous_midpoint=0,
                    color_continuous_scale="RdBu",
                    )
    
    tickvals = np.arange(patterns.shape[2])
    ticktext = token_df["unique_token"].tolist()
    
    # add tokens as x-ticks and y-ticks, for each facet
    # Update x-ticks and y-ticks for each facet
    for i in range(len(facet_col_labels)):
        fig.update_xaxes(
            dict(tickmode='array', tickvals=tickvals, ticktext=ticktext),
            row=1, col=i+1
        )
        fig.update_yaxes(
            dict(tickmode='array', tickvals=tickvals, ticktext=ticktext),
            row=1, col=i+1
        )
    
    
    # add facet col labels:
    for i, label in enumerate(facet_col_labels):
        fig.layout.annotations[i].text = label
        fig.layout.annotations[i].font.size = 20
        
    fig.update_layout(
        width=1200,
        height=800,
    )
    fig.show()



HEAD_IDX = 5
LAYER_IDX = sparse_autoencoder.cfg.hook_point_layer
patterns_original = original_cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
# plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")
patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")

In [ ]:
POS_INTEREST = 10
UNIQUE_TOKEN_INTEREST = token_df["unique_token"][POS_INTEREST]
feature_acts_of_interest = feature_acts[POS_INTEREST]
title = "test"

# plot_line_with_top_10_labels(feature_acts_of_interest, "", 25)

top_k_feature_inds = (feature_acts[1:] > 0).sum(dim=0).nonzero().squeeze()

features_acts_by_token_df = pd.DataFrame(
    feature_acts[:,top_k_feature_inds[:]].detach().cpu().T,
    index = [f"feature_{i}" for i in top_k_feature_inds.flatten().tolist()],
    columns = token_df["unique_token"])

tmp = features_acts_by_token_df.sort_values(UNIQUE_TOKEN_INTEREST, ascending=False).T
px.line(tmp, 
        title=f"{title}: Features Activation by Token in Prompt", 
        color_discrete_sequence=px.colors.qualitative.Plotly,
        height=400).show()

px.line(tmp + 1e-4,
        log_y=True, 
        title=f"{title}: Features Activation by Token in Prompt", 
        color_discrete_sequence=px.colors.qualitative.Plotly,
        height=400).show()


In [ ]:
# render_feature_dashboard(16263)
# render_feature_dashboard(5673)

### intervene and calculate attn

In [ ]:
original_cache.keys()

In [ ]:
def feature_ablation_effect_on_attn_experiment(
    tokens,
    token_df,
    original_cache,
    cache_reconstructed_query,
    key_pos=None,
    query_pos=-1,
    layer_idx=sparse_autoencoder.cfg.hook_point_layer,
    head_idx=5,
    pattern_or_score = "pattern",
):
    """
    if key_pos is unspecified, intervene at query pos.
    else:
        intervene at key pos.


    """

    if pattern_or_score == "pattern":
        hook_text = "hook_pattern"
    else:
        hook_text = "hook_attn_scores"
        
    token_df = make_token_df(model, tokens)
    patterns = (
        original_cache[f"blocks.{layer_idx}.attn.{hook_text}"][0, head_idx]
        .detach()
        .cpu()
    )
    token_df["original_attn"] = patterns[query_pos,]
    patterns = (
        cache_reconstructed_query[f"blocks.{layer_idx}.attn.{hook_text}"][0, head_idx]
        .detach()
        .cpu()
    )
    token_df["reconstructed_attn"] = patterns[query_pos,]

    if key_pos is not None:
        intervene_pos = key_pos
    else:
        intervene_pos = query_pos

    head_hook_query_name = utils.get_act_name("q", layer_idx)

    vals, inds = torch.topk(feature_acts[intervene_pos], 35)
    original_act = original_cache[sparse_autoencoder.cfg.hook_point]

    for feature in inds:
        features_to_remove = [feature]
        sae_out, _, _, mse_loss, _ = sparse_autoencoder(original_act)

        # need to generate query
        def replacement_hook(resid_pre, hook, new_resid_pre=sae_out):
            return new_resid_pre

        with model.hooks(
            fwd_hooks=[
                (
                    utils.get_act_name(
                        "resid_pre", sparse_autoencoder.cfg.hook_point_layer
                    ),
                    replacement_hook,
                )
            ]
        ):
            _, resid_pre_cache = model.run_with_cache(
                tokens, return_type="loss", loss_per_token=True
            )
            sae_out = resid_pre_cache[head_hook_query_name][:, :, head_idx]

        def remove_feature_hook(
            hook_in, hook, position=intervene_pos, features_to_remove=features_to_remove
        ):
            for feature_to_remove in features_to_remove:
                # print(feature_acts[0,position,feature_to_remove].item())
                feature_dir = (
                    feature_acts[position, feature_to_remove]
                    * sparse_autoencoder.W_dec[feature_to_remove]
                )
                hook_in -= feature_dir
            return hook_in

        with model.hooks(
            fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_feature_hook)]
        ):
            _, cache_removed_feature = model.run_with_cache(
                tokens, return_type="loss", loss_per_token=True
            )

        patterns = (
            cache_removed_feature[f"blocks.{layer_idx}.attn.hook_pattern"][0, head_idx]
            .detach()
            .cpu()
        )
        token_df[f"ablated_feature_{feature}"] = patterns[query_pos,]
        
    # then do the same with original act - sae_out
    sae_out, _, _, mse_loss, _ = sparse_autoencoder(original_act)
    dummy_feature = original_act - sae_out

    def replacement_hook(resid_pre, hook, new_resid_pre=sae_out):
        return new_resid_pre

    with model.hooks(
        fwd_hooks=[
            (
                utils.get_act_name(
                    "resid_pre", sparse_autoencoder.cfg.hook_point_layer
                ),
                replacement_hook,
            )
        ]
    ):
        _, resid_pre_cache = model.run_with_cache(
            tokens, return_type="loss", loss_per_token=True
        )
        sae_out = resid_pre_cache[head_hook_query_name][:, :, head_idx]

    def remove_feature_hook(
        hook_in, hook, position=intervene_pos, features_to_remove=features_to_remove
    ):
        hook_in -= dummy_feature
        return hook_in

    with model.hooks(
        fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_feature_hook)]
    ):
        _, cache_removed_feature = model.run_with_cache(
            tokens, return_type="loss", loss_per_token=True
        )

    patterns = (
        cache_removed_feature[f"blocks.{layer_idx}.attn.hook_pattern"][0, head_idx]
        .detach()
        .cpu()
    )
    token_df[f"ablated_sae_residual"] = patterns[query_pos,]

    return token_df


query_pos = 10
tokens = torch.concat(
    [torch.tensor(model.tokenizer.eos_token_id).unsqueeze(0), random_tokens.cpu()],
    dim=0,
).unsqueeze(0)
test_query = feature_ablation_effect_on_attn_experiment(
    tokens,
    token_df,
    original_cache,
    cache_reconstructed_query,
    key_pos=None,
    query_pos=query_pos,
    layer_idx=sparse_autoencoder.cfg.hook_point_layer,
    head_idx=5,
    pattern_or_score="pattern"
)

fig = px.line(
    test_query,
    x="unique_token",
    y=["original_attn", "reconstructed_attn", "ablated_sae_residual"]
    + [i for i in test_query.columns if "ablated_feature" in i],
    hover_name="str_tokens",
    hover_data=["pos", "batch", "label"],
    title="Original vs Reconstructed attention",
    width=1000,
)

fig.show()

key_pos = 5
query_pos = 10
tokens = torch.concat(
    [torch.tensor(model.tokenizer.eos_token_id).unsqueeze(0), random_tokens.cpu()],
    dim=0,
).unsqueeze(0)
test_key = feature_ablation_effect_on_attn_experiment(
    tokens,
    token_df,
    original_cache,
    cache_reconstructed_query,
    key_pos=key_pos,
    query_pos=query_pos,
    layer_idx=sparse_autoencoder.cfg.hook_point_layer,
    head_idx=5,
    pattern_or_score="pattern"
)

fig = px.line(
    test_key,
    x="unique_token",
    y=["original_attn", "reconstructed_attn", "ablated_sae_residual"]
    + [i for i in test_key.columns if "ablated_feature" in i],
    hover_name="str_tokens",
    hover_data=["pos", "batch", "label"],
    title="Original vs Reconstructed attention",
    width=1000,
)

fig.show()

## Virtual Weight Stuff

#### Norms

In [ ]:
layer = sparse_autoencoder.cfg.hook_point_layer
W_QK_5 = torch.stack(
    [model.W_Q[layer,i] @ model.W_K[layer,i].T for i in range(12)],
    dim=0
).cpu()
W_QK_5.norm(dim=(1,2))

In [ ]:


def get_feature_weight_proj(W_dec_features, feature_indices, sparse_autoencoder=sparse_autoencoder, model = model):
    W_dec_features = W_dec_features.cpu() - W_dec_features.cpu().mean(dim=0, keepdim=True)
    layer = sparse_autoencoder.cfg.hook_point_layer
    W_QK_5 = torch.stack(
        [model.W_Q[layer,i] @ model.W_K[layer,i].T for i in range(12)],
        dim=0
    ).cpu()
    W_QK_5 = W_QK_5 / W_QK_5.norm(dim=(1,2), keepdim=True)

    query_proj_dec_weights = W_dec_features @ (W_QK_5 - W_QK_5.mean(dim=0, keepdim=True))
    query_proj_dec_weights_df = pd.DataFrame(
        query_proj_dec_weights.norm(dim=-1).detach().cpu().numpy().T,
        columns = [f"L{layer}H" + str(i) for i in range(12)],
        index = [f"feature_{i}" for i in feature_indices])



    key_proj_dec_weights = W_dec_features @ W_QK_5.transpose(2,1)
    key_proj_dec_weights_df = pd.DataFrame(
        key_proj_dec_weights.norm(dim=-1).detach().cpu().numpy().T,
        columns = [f"L{layer}H" + str(i) for i in range(12)],
        index = [f"feature_{i}" for i in feature_indices])
    
    
    return query_proj_dec_weights_df, key_proj_dec_weights_df



query_proj_dec_weights_df, key_proj_dec_weights_df = get_feature_weight_proj(sparse_autoencoder.W_dec, range(sparse_autoencoder.cfg.d_sae))
query_proj_dec_weights_df["log_feature_sparsity"] = log_feature_sparsity
key_proj_dec_weights_df["log_feature_sparsity"] = log_feature_sparsity

layer = sparse_autoencoder.cfg.hook_point_layer
head_list = [f"L{layer}H" + str(i) for i in range(12)]
max_prox = max(query_proj_dec_weights_df.max().max(), key_proj_dec_weights_df.max().max())

px.histogram(
    query_proj_dec_weights_df[query_proj_dec_weights_df["log_feature_sparsity"]>-9],
    x=head_list,
    barmode='overlay',
    title = "Query projection onto decoder weights",
    width = 1000,
    height = 600,
    nbins=1000,
    range_x=[0, max_prox]
).show()

px.histogram(
    key_proj_dec_weights_df[key_proj_dec_weights_df["log_feature_sparsity"]>-9],   
    x=head_list,    
    barmode='overlay',
    title = "Key projection onto decoder weights",
    width = 1000,
    height = 600,
    nbins =1000,
    range_x=[0, max_prox]
).show()


In [ ]:
all_proj_dec_weights_df = query_proj_dec_weights_df.join(key_proj_dec_weights_df, how="inner", lsuffix="_query", rsuffix="_key")
all_proj_dec_weights_df["log_feature_sparsity"] = log_feature_sparsity

for head in [5]:
    px.scatter(
        all_proj_dec_weights_df[all_proj_dec_weights_df["log_feature_sparsity"]>-9],
        x=f"L5H{head}_query",
        y=f"L5H{head}_key",
        color = "log_feature_sparsity",
        title = f"Norm of Query vs Key projection onto decoder weights (head {head})",
        height = 1000,
        width = 1500,
        hover_name=all_proj_dec_weights_df[all_proj_dec_weights_df["log_feature_sparsity"]>-9].index,
        marginal_x="histogram",
        marginal_y="histogram",
        opacity=0.5,
    ).show()
# px.parallel_coordinates(
#     all_proj_dec_weights_df[all_proj_dec_weights_df["log_feature_sparsity"]>-9],
#     dimensions=["L5H5_query", "L5H5_key", "log_feature_sparsity"],
#     title = "Query vs Key projection onto decoder weights",
#     height = 1000,
#     width = 1500,
# ).show()


In [ ]:
px.bar(torch.svd(sparse_autoencoder.W_dec[log_feature_sparsity>-9].cpu())[1])

In [ ]:
px.bar(model.b_Q[5,:].cpu().norm(dim=-1))

In [ ]:
px.bar(model.b_K[5,:].cpu().norm(dim=-1))

In [ ]:
all_proj_dec_weights_df[all_proj_dec_weights_df["log_feature_sparsity"]>-9].sort_values("L5H5_query", ascending=False).head(10).style.background_gradient(
    cmap="coolwarm", axis=0)
for feature in all_proj_dec_weights_df[all_proj_dec_weights_df["log_feature_sparsity"]>-9].sort_values("L5H5_query", ascending=False).head(10).index:
    render_feature_dashboard(feature.split("_")[1])

In [ ]:
all_proj_dec_weights_df[all_proj_dec_weights_df["log_feature_sparsity"]>-9].sort_values("L5H5_key", ascending=False).head(10).style.background_gradient(
    cmap="coolwarm", axis=0)
for feature in all_proj_dec_weights_df[all_proj_dec_weights_df["log_feature_sparsity"]>-9].sort_values("L5H5_key", ascending=False).head(10).index:
    render_feature_dashboard(feature.split("_")[1])

In [ ]:
display(all_proj_dec_weights_df.sort_values("L5H5_query", ascending=False).head(30)[["L5H5_query", "L5H5_key", "log_feature_sparsity"]].style.background_gradient(
    cmap="coolwarm", axis=0))

top_query_features = all_proj_dec_weights_df.sort_values("L5H5_query", ascending=False).head(30).index
top_query_features = [int(i.split("_")[1]) for i in top_query_features]

for feature in top_query_features:
    render_feature_dashboard(feature)

In [ ]:
display(all_proj_dec_weights_df.sort_values("L5H5_key", ascending=False).head(30)[["L5H5_query", "L5H5_key", "log_feature_sparsity"]].style.background_gradient(
    cmap="coolwarm", axis=0))

top_query_features = all_proj_dec_weights_df.sort_values("L5H5_key", ascending=False).head(30).index
top_query_features = [int(i.split("_")[1]) for i in top_query_features]

for feature in top_query_features:
    render_feature_dashboard(feature)

#### Bipartite Graph

In [ ]:
W_QK_5_5 = model.W_Q[5,5].cpu() @ model.W_K[5,5].T.cpu()
expanded_qk_W_dec = sparse_autoencoder.W_dec.cpu() @ W_QK_5_5 @ sparse_autoencoder.W_dec.T.cpu()
expanded_qk_W_dec_alive = expanded_qk_W_dec[log_feature_sparsity>-9][:,log_feature_sparsity>-9]
expanded_qk_W_dec_alive.shape

In [ ]:
px.histogram(expanded_qk_W_dec_alive.flatten()[torch.randperm(expanded_qk_W_dec_alive.flatten().shape[0])[:10000]], nbins=1000, title="expanded_qk_W_dec_alive")

In [ ]:
flattened_qk_W_dec = expanded_qk_W_dec_alive.flatten().cpu()
ids_flattend = np.arange(flattened_qk_W_dec.shape[0])
ids_flattend = ids_flattend[flattened_qk_W_dec.abs()>0.1]
query_ids = ids_flattend // flattened_qk_W_dec.shape[0]
key_ids = ids_flattend % flattened_qk_W_dec.shape[0]
tmp = pd.DataFrame(
   [flattened_qk_W_dec, query_ids, key_ids],
    index=["val", "query", "key"]
).T
tmp = tmp.sort_values("val", ascending=False)
tmp.head()

In [ ]:
flattened_qk_W_dec = expanded_qk_W_dec_alive.flatten().cpu()
query_id = np.array(range(len(flattened_qk_W_dec))) // expanded_qk_W_dec_alive.shape[1]
key_id = np.array(range(len(flattened_qk_W_dec))) % expanded_qk_W_dec_alive.shape[1]
query_key_df = pd.DataFrame(dict(query_id=query_id, key_id=key_id, val=flattened_qk_W_dec))
query_key_df= query_key_df[query_key_df["val"].abs()>0.05]

#### try to get a sparse graph

In [ ]:
import networkx as nx
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

edge_weights = expanded_qk_W_dec_alive
edge_weights[edge_weights.abs()<0.3] = 0
print(edge_weights.nonzero().shape)
# convert to numpy sparse matrix
edge_weights = csr_matrix(edge_weights.detach().cpu().numpy())
n_components, labels = connected_components(csgraph=edge_weights, directed=True, return_labels=True)
n_components

In [ ]:
(log_feature_sparsity>-9).nonzero().squeeze()

In [ ]:
tmp = pd.DataFrame(
    [labels, log_feature_sparsity[log_feature_sparsity>-9].tolist(), (log_feature_sparsity>-9).nonzero().squeeze().tolist()],
    index=["component", "log_feature_sparsity", "feature_id"]).T

# group by connected component 
tmp.groupby("component").feature_id.count()

In [ ]:
labels.shape

In [ ]:
key_embeddings = sparse_autoencoder.W_dec.cpu() @ model.W_K[5,5].cpu()
key_feature_labels = [f"key_feature_{i}" for i in range(key_embeddings.shape[0])]
print(key_embeddings.shape)

query_embeddings = sparse_autoencoder.W_dec.cpu() @ model.W_Q[5,5].cpu()
query_feature_labels = [f"query_feature_{i}" for i in range(query_embeddings.shape[0])]
print(query_embeddings.shape)

# filter out features which are dead
key_embeddings = key_embeddings[log_feature_sparsity>-9]
query_embeddings = query_embeddings[log_feature_sparsity>-9]
key_feature_labels = [key_feature_labels[i] for i in (log_feature_sparsity>-9).nonzero().squeeze().tolist()]
query_feature_labels = [query_feature_labels[i] for i in (log_feature_sparsity>-9).nonzero().squeeze().tolist()]


##### Random Filter + Neighbours (not super great)

In [ ]:
# let's see a filter around a 

all_features_we_care_about = []

feature = 86
vals, inds = torch.topk(expanded_qk_W_dec[feature], 10)
all_features_we_care_about += inds.tolist()

for feature in inds:
    vals, inds = torch.topk(expanded_qk_W_dec[:, feature], 10)
    all_features_we_care_about += inds.tolist()
    
all_features_we_care_about = list(set(all_features_we_care_about))
print(len(all_features_we_care_about))


tmp = pd.DataFrame(expanded_qk_W_dec[all_features_we_care_about,:][:,all_features_we_care_about],
                   index = [f"feature_{i}" for i in all_features_we_care_about],
                     columns = [f"feature_{i}" for i in all_features_we_care_about])

px.imshow(tmp, color_continuous_midpoint=0, color_continuous_scale="RdBu", height = 800,
          range_color=[-1,1]).show()

## UMAP

In [ ]:
key_embeddings = (sparse_autoencoder.W_dec.cpu() @ model.W_K[5,5].cpu()) + model.b_K[5,5].cpu()
key_embeddings = key_embeddings - key_embeddings.mean(dim=0)
key_feature_labels = [f"key_feature_{i}" for i in range(key_embeddings.shape[0])]
print(key_embeddings.shape)

query_embeddings = (sparse_autoencoder.W_dec.cpu() @ model.W_Q[5,5].cpu()) + model.b_Q[5,5].cpu()
query_embeddings = query_embeddings - query_embeddings.mean(dim=0)
query_feature_labels = [f"query_feature_{i}" for i in range(query_embeddings.shape[0])]
print(query_embeddings.shape)

# filter out features which are dead
key_embeddings = key_embeddings[log_feature_sparsity>-9]
query_embeddings = query_embeddings[log_feature_sparsity>-9]
key_feature_labels = [key_feature_labels[i] for i in (log_feature_sparsity>-9).nonzero().squeeze().tolist()]
query_feature_labels = [query_feature_labels[i] for i in (log_feature_sparsity>-9).nonzero().squeeze().tolist()]


all_embeddings = torch.cat([key_embeddings, query_embeddings], dim=0)
print(all_embeddings.shape)
all_embeddings_labels = key_feature_labels + query_feature_labels
print(all_embeddings_labels[:10])

In [ ]:
px.histogram(key_embeddings.norm(dim=-1), nbins=1000, title="key_embeddings", width = 500).show()
px.histogram(query_embeddings.norm(dim=-1), nbins=1000, title="key_embeddings", width = 500).show()

In [ ]:
import umap 
import hdbscan

reducer = umap.UMAP(
    n_neighbors=30,
    min_dist=0.0001,
    n_components=2,
    metric="cosine",
    random_state=42,
)

all_embeddings_normalized = all_embeddings / all_embeddings.norm(dim=-1).unsqueeze(1)
ummap_result = reducer.fit_transform(all_embeddings_normalized.numpy())


In [ ]:
from sklearn.manifold import trustworthiness

n_neighbors_list = 3 + np.arange(100)
trust_scores = []
for n_neighbors in n_neighbors_list:
    random_indices = np.random.permutation(all_embeddings.shape[0])[:1000]
    trust = trustworthiness(
        all_embeddings_normalized[random_indices].numpy(),
        ummap_result[random_indices],
        n_neighbors=n_neighbors,
        metric="cosine",
    )
    trust_scores.append(trust)
    
px.line(x= n_neighbors_list, y = trust_scores, title="Trustworthiness of UMAP embeddings", width=500).show()

In [ ]:
from sklearn.neighbors import NearestNeighbors

def knn_overlap(X_high, X_low, n_neighbors=5):
    nn_high = NearestNeighbors(n_neighbors=n_neighbors).fit(X_high)
    nn_low = NearestNeighbors(n_neighbors=n_neighbors).fit(X_low)

    _, indices_high = nn_high.kneighbors(X_high)
    _, indices_low = nn_low.kneighbors(X_low)

    overlap = np.mean([len(np.intersect1d(indices_high[i], indices_low[i])) for i in range(X_high.shape[0])])
    return overlap / n_neighbors

overlap_score = knn_overlap(all_embeddings_normalized.numpy(), ummap_result, n_neighbors=3)
overlap_score

In [ ]:

umap_df = pd.DataFrame(ummap_result, columns=["ummap_x", "ummap_y"])
umap_df["feature_sparsity"] = log_feature_sparsity[log_feature_sparsity>-9].detach().cpu().numpy().tolist() + log_feature_sparsity[log_feature_sparsity>-9].detach().cpu().numpy().tolist()
umap_df["label"] = all_embeddings_labels
umap_df["key_or_query"] = ["key"]*len(key_feature_labels) + ["query"]*len(query_feature_labels)

clusterer = hdbscan.HDBSCAN(min_cluster_size=10)
clusterer.fit(ummap_result)
umap_df["cluster"] = clusterer.labels_
umap_df["cluster"] = umap_df["cluster"].astype(str)
umap_df["feature_sparsity"] = log_feature_sparsity[log_feature_sparsity>-9].detach().cpu().numpy().tolist() + log_feature_sparsity[log_feature_sparsity>-9].detach().cpu().numpy().tolist()
umap_df["label"] = all_embeddings_labels
umap_df["key_or_query"] = ["key"]*len(key_feature_labels) + ["query"]*len(query_feature_labels)

fig = px.scatter(
        umap_df,
        x="ummap_x",
        y="ummap_y",
        color="key_or_query",
        # color="feature_sparsity",
        # color="cluster",
        color_continuous_midpoint=-2,
        color_continuous_scale="RdBu",
        hover_name="label",
        hover_data=["feature_sparsity", "cluster", "key_or_query"],
        # opacity=0.1,
        template="plotly",
    )

# don't show legend
# fig.update_layout(showlegend=False)

# make points larger
fig.update_traces(marker=dict(size=4))

# make it wide and tall
fig.update_layout(height=1200, width=1600)
fig.show()

In [ ]:
feature_label = "query_feature_18"
query_embedding_feature_normalized = query_embeddings[query_feature_labels.index(feature_label)] / query_embeddings[query_feature_labels.index(feature_label)].norm()
umap_df["cosine_distance_with_query_feature_18"] = 1 - (all_embeddings_normalized @ query_embedding_feature_normalized).detach().cpu().numpy()
feature_index_in_all_embeddings = query_feature_labels.index(feature_label) + len(key_feature_labels) if "query" in feature_label else key_feature_labels.index(feature_label)
umap_df["umap_distance_from_query_feature_18"] = np.linalg.norm((ummap_result - ummap_result[feature_index_in_all_embeddings]), axis=1)
# umap_df.sort_values("distance_from_query_feature_18", ascending=True).head(10)


display(umap_df[(umap_df.label == "key_feature_86") | (umap_df.label == "query_feature_18")])

In [ ]:

# get the spearman rank correlation between cosine distance and umap distance
from scipy.stats import spearmanr
print(spearmanr(umap_df["cosine_distance_with_query_feature_18"], umap_df["umap_distance_from_query_feature_18"]))

px.scatter(
    umap_df,
    x="cosine_distance_with_query_feature_18",
    y="umap_distance_from_query_feature_18",
    color = "key_or_query",
    color_continuous_midpoint=0,
    color_continuous_scale="RdBu",
    hover_name="label",
    hover_data=["feature_sparsity", "cluster", "key_or_query"],
    template="plotly",
    marginal_x="histogram",
    marginal_y="histogram",
    height=1000,
    width=1000,
).show()


In [ ]:
umap_df.sort_values("cosine_distance_with_query_feature_18", ascending=True).query("key_or_query=='key'").head(10).style.background_gradient(
    cmap="coolwarm", axis=0)

In [ ]:
# pick a cluster and sample the dashboards
cluster_of_interest = 913
cluster_examples = tmp_df[tmp_df["cluster"]==str(cluster_of_interest)].sample(5)
display(cluster_examples)
for feature in cluster_examples["label"]:
    render_feature_dashboard(feature.split("_")[2])